In [1]:
!pip install --upgrade pip
!apt update
!apt install sudo
!sudo apt-get install python3.10-dev -y
!pip install sentence_transformers pandas
!pip install torch==2.0.1 torchvision==0.15.2 torchaudio==2.0.2 --index-url https://download.pytorch.org/whl/cu118


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 3.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 21.7 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.8/10.8 MB 71.6 MB/s eta 0:00:0000:010:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 36.4/36.4 MB 47.3 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 85.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.2/302.2 kB 71.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 670.2/670.2 MB 3.6 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 59.7 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 93.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 75.0 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 731.7/731.7 MB 3.3 MB/s eta 0:00:0000:0100:01
     

In [2]:
import os 
os.chdir('/home2/eunsik12/Sbert')
os.getcwd()

'/home2/eunsik12/Sbert'

In [37]:
import pandas as pd

train_df = pd.read_csv('./질문_업무_db.csv',  encoding='utf-8', index_col=[0]) #cp949
train_df.head(1)

,접수번호,업무,질문
0,220101032533665,상수도사업본부 중부수도사업소 요금과 요금1팀중구용산구 부서의 업무는 차량관리 69오...,수도 요금 누수 감액을 받았는데 장기간 누수가 의심되어 문의


In [4]:
train_df.loc[1]['업무'], train_df.loc[1]['질문']

('행정국 인력개발과 교육팀 부서의 업무는 직장교육 멘토링 포함 운영에 관한 사항 학습동아리 운영 및 연구 저술 공무원 지원에 관한 사항 독도아카데미에 관한 사항 대학생 아르바이트 운영 인턴십 경력증명 발급 등 5급 승진자 교육 관련 사항 9급 신임자 교육 관리 공무직 교육 관련 사항 자발적 학습조직 활성화 추진 지원 대직자 이하진',
 '서울시 대학생 아르바이트 선발 결과 문의')

In [46]:
import pandas as pd
import torch

class sbert_dataset(torch.utils.data.Dataset):
    def __init__(self, df):
        self.df = df
        
    def __len__(self, ):
        return len(self.df)
    
    def __getitem__(self, idx):
        try:
            # if idx < 0 or idx >= len(self.df):
            #     raise IndexError(f"Index {idx} out of range for length {len(self.df)}")
            if type(idx) == int:
                return [self.df.loc[idx, '질문'], self.df.loc[idx, '업무']]
            return [[a,b] for a, b in zip(self.df.loc[idx, '질문'].tolist(), self.df.loc[idx, '업무'].tolist())]

        except KeyError as e:
            raise IndexError()
train_df.dropna(inplace=True)
train_df.reset_index(drop=True, inplace=True)
train_dataset = sbert_dataset(train_df)

In [47]:
from sentence_transformers import SentenceTransformer, InputExample, losses,  evaluation, models
from torch.utils.data import DataLoader

def make_sts_input_example(dataset):
    input_examples = []
    for i, data in enumerate(dataset):
        sentence1 = data[0]
        sentence2 = data[1]
        score = 1.
        input_examples.append(InputExample(texts=[sentence1, sentence2], label=score))

    return input_examples
train_dataset = make_sts_input_example(train_dataset)

In [58]:
# word_embedding_model = models.Transformer('bert-base-uncased', max_seq_length=256)
# pooling_model = models.Pooling(word_embedding_model.get_word_embedding_dimension())

# model = SentenceTransformer(modules=[word_embedding_model, pooling_model], device ='cuda')


#Define your train dataset, the dataloader and the train loss
model = SentenceTransformer('snunlp/KR-SBERT-V40K-klueNLI-augSTS')
train_dataloader = DataLoader(train_dataset, shuffle=True, batch_size=16)
train_loss = losses.CosineSimilarityLoss(model)

sentences1 = ['상수도사업본부 중부수도사업소 요금과 요금1팀중구용산구 부서의 업무는 차량관리 69오6382 팀 휴대폰 관리 010 6428 요금심사 및 체납징수 신당2동 신당4동 청구동 신당3동 남영동', 
              '행정국 인력개발과 교육팀 부서의 업무는 직장교육 멘토링 포함 운영에 관한 사항 학습동아리 운영 및 연구 저술 공무원 지원에 관한 사항 독도아카데미에 관한 사항 대학생 아르바이트 운영 인턴십 경력증명 발급 등 5급 승진자 교육 관련 사항 9급 신임자 교육 관리 공무직 교육 관련 사항 자발적 학습조직 활성화 추진 지원 대직자 이하진']
sentences2 = ['수도 요금 누수 감액을 받았는데 장기간 누수가 의심되어 문의', '서울시 대학생 아르바이트 선발 결과 문의']
scores = [0.9, 0.99]

evaluator = evaluation.EmbeddingSimilarityEvaluator(sentences1, sentences2, scores)

In [59]:
model.fit(train_objectives=[(train_dataloader, train_loss)], epochs=1, warmup_steps=100, evaluator=evaluator, evaluation_steps=10)

Epoch: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [1:00:17<00:00, 3617.22s/it]


In [62]:
print(model)

SentenceTransformer(
  (0): Transformer({'max_seq_length': 128, 'do_lower_case': False}) with Transformer model: BertModel 
  (1): Pooling({'word_embedding_dimension': 768, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False})
)


In [103]:
from sentence_transformers import SentenceTransformer, util
original_model = SentenceTransformer('snunlp/KR-SBERT-V40K-klueNLI-augSTS')

vec1 = original_model.encode('미치겠네 이거', show_progress_bar=True, batch_size=32)
vec2 = original_model.encode(sentences2[0], show_progress_bar=True, batch_size=32)

util.cos_sim(vec1.reshape(1,-1), vec2.reshape(1,-1))

Batches: 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00, 117.90it/s]


tensor([[0.1840]])

In [118]:
from sentence_transformers import SentenceTransformer, util

vec1 = model.encode('어떻게 이럴수가 있지?', show_progress_bar=True, batch_size=32)
vec2 = model.encode(sentences2[0], show_progress_bar=True, batch_size=32)

util.cos_sim(vec1.reshape(1,-1), vec2.reshape(1,-1))

Batches: 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00, 116.80it/s]


tensor([[1.0000]])

In [119]:
vec1[:20], vec2[:20]

(array([-0.0187882 ,  0.24176589, -0.09130772,  0.10925875,  0.16705008,
         0.5254224 , -0.36401936,  0.13973144,  0.09008235, -0.14720711,
         0.23667361, -0.1609391 ,  0.06406453,  0.06540694,  0.074884  ,
        -0.44693372, -0.05529083, -0.5294041 , -0.41722044,  0.07923052],
       dtype=float32),
 array([-0.01609456,  0.24628271, -0.09392784,  0.10840915,  0.17545818,
         0.5303293 , -0.36541831,  0.13841566,  0.08577701, -0.14273088,
         0.2314083 , -0.15272026,  0.06756931,  0.06899818,  0.06411265,
        -0.45060453, -0.05140609, -0.522928  , -0.42235118,  0.08448441],
       dtype=float32))

In [72]:
model.save('ver1')

In [113]:
from sklearn.metrics.pairwise import cosine_similarity
cosine_similarity(vec1.reshape(1,-1), vec2.reshape(1,-1))

array([[0.9999032]], dtype=float32)

In [81]:
train_df

,index,접수번호,업무,질문
0,0,220101032533665,상수도사업본부 중부수도사업소 요금과 요금1팀중구용산구 부서의 업무는 차량관리 69오...,수도 요금 누수 감액을 받았는데 장기간 누수가 의심되어 문의
1,1,220101038863665,행정국 인력개발과 교육팀 부서의 업무는 직장교육 멘토링 포함 운영에 관한 사항 학습...,서울시 대학생 아르바이트 선발 결과 문의
2,2,220102056263873,도시교통실 교통기획관 교통지도과 시스템운영관리팀 부서의 업무는 무인단속시스템 CCT...,구로구 불법주정차 신고건 조치 안됨 문의
3,3,220103021953646,도시교통실 교통기획관 택시정책과 택시면허팀 부서의 업무는 개인택시 인허가 및 운영 ...,서울시 개인택시 등록대수 확인 문의
4,4,220103023183091,재무국 세무과 부동산세팀 부서의 업무는 부동산 취득세 과징업무 총괄 및 운영 지원 ...,서울시 부동산 취득세 문의
...,...,...,...,...
416104,416534,201231110833541,주차지도1팀 부서의 업무는 불법 주정차 순찰 및 단속업무,재택 송파구 주정차 단속 민원 석촌동 견인요청 추가 요청
416105,416535,201231120783825,면목본동 행정팀 부서의 업무는 청원경찰,코로나 관련 가정집에서 인 이상 집합금지 신고한다고 하심
416106,416536,201231122093825,주차지도1팀 부서의 업무는 불법 주정차 단속 종합계획 수립 팀예산 결산 주요 업무계...,불법주정차 단속되어 과태료 문의
416107,416537,201231122573295,공중위생민원팀 부서의 업무는 식품 및 공중위생 영업신고,중구 음식점 코로나로인한 사회적 거리두기 방법문의


In [92]:
int(len(train_df) * 0.1)

41610

In [134]:
from torch.utils.data import DataLoader
from sentence_transformers import losses
from tqdm import tqdm


tqdm.pandas()
train_df['input'] = train_df.progress_apply(lambda x: InputExample(texts=[x.질문, x.업무]), axis = 1)
test_df = train_df[-41610:].copy()
train_df = train_df[:-41610] 
train_examples = list(train_df['input'])
test_examples = list(test_df['input'])

train_dataloader = DataLoader(train_examples, shuffle=True, batch_size=16)
test_dataloader = DataLoader(test_examples, shuffle=True, batch_size=16)

train_loss = losses.MultipleNegativesRankingLoss(model=model)


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 332889/332889 [00:08<00:00, 41394.66it/s]


In [133]:
evaluator = evaluation.EmbeddingSimilarityEvaluator(test_df['업무'].to_list(), test_df['질문'].to_list(), [1] * len(test_df))
evaluator(model)

/usr/local/lib/python3.10/dist-packages/scipy/stats/_stats_py.py:4781: ConstantInputWarning: An input array is constant; the correlation coefficient is not defined.
  warnings.warn(stats.ConstantInputWarning(msg))
/usr/local/lib/python3.10/dist-packages/scipy/stats/_stats_py.py:5445: ConstantInputWarning: An input array is constant; the correlation coefficient is not defined.
  warnings.warn(stats.ConstantInputWarning(warn_msg))


nan

In [135]:
model = SentenceTransformer('snunlp/KR-SBERT-V40K-klueNLI-augSTS', device= 'cuda')
model.fit(train_objectives=[(train_dataloader, train_loss)], epochs=1, warmup_steps=1000, show_progress_bar = False)

In [132]:
vec1 = model.encode(sentences1[1], show_progress_bar=True, batch_size=32)
vec2 = model.encode(sentences2[1], show_progress_bar=True, batch_size=32)

util.cos_sim(vec1.reshape(1,-1), vec2.reshape(1,-1))

Batches: 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00, 121.04it/s]


tensor([[0.2490]])